In [ ]:
from algo_gen import algo_genetique
from algo_four import algo_fourmis
import numpy as np
from random import randint
from time import time
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
# Chargement de la matrice adjacence à partir d'un fichier
def load_matrix():
    mat = []
    with open('mat_adjacence', 'r') as file:   
        for l in file.readlines():  
            val = [int(x) for x in l.split()]
            mat.append(val)            
    return mat

# Créer une matrice d'adjacence dont le cout du circuit Hamiltonien optimal est celui de la périphérie  
def create_matrix(n_ville):
    mat = np.zeros((n_ville, n_ville))
    
    for i in range(n_ville):
        for j in range(i, n_ville):
            cout = randint(2, 10)
            if i == j:
                cout = 0
            i_after = i+1 if i+1 < n_ville else 0
            if i_after == j:
                cout = 1
                 
            mat[i][j] = cout
            mat[j][i] = cout
    mat[n_ville-1][0] = 1
    mat[0][n_ville-1] = 1

    return mat
 

In [ ]:
algo_fourmis(create_matrix(7), verbal=True)

In [ ]:
algo_genetique(create_matrix(7), time_max=100, verbal=True)

In [ ]:
def test_acc(algo, mat_size = 5, n_try=1, time_max=100): 
    """
    Test l'algorithme d'exploration avec des matrice dont le chemin au cout optimal correspond à la périphérie, et retourne l'accuracy moyen
        mat_size : taille de la matrice d'adjacence qui sera exploré
        n_try : Nombre de test effectué
    """
    acc = 0 
    # On test plusieurs fois et on obtient
    for _ in range(n_try):
        cost_mat = create_matrix(mat_size)
        # Accuracy = meilleur chemin / chemin trouvé par l'algo 
        acc += mat_size / algo(cost_mat, time_max=time_max, verbal=False)[1] # en sachant que par construction: Taille optimal = taille de matrice    
        
    return acc / n_try

In [ ]:
def main_test(algorithmes, n_city_start = 4, n_city_end = 10): 
    """
    Effectue des tests sur une liste d'algorithme, et affiche sous la forme de diagrame l'évolution des tests (avec precision et temps d'execution)
    algorithmes : Liste des fonctions avec comme paramètre la matrice de coût et le temps optimal
        Chacune de ses fonctions doivent retourné le chemins optimal et le coût sous la forme d'un tuple.
    """
    
    accuracies = [[] for _ in algorithmes]
    times = [[] for _ in algorithmes]  
    mat_sizes = [[] for _ in algorithmes]
     
     
    for size in range(n_city_start, n_city_end, 1): 
        for i, algo in enumerate(algorithmes):
            start_time = time()
            accuracies[i].append(test_acc(algo=algo, mat_size=size, n_try=100, time_max=50))
            times[i].append(time()-start_time) 
            
        fig, ax = plt.subplots(len(algorithmes))
        if len(algorithmes) == 1:
            ax = np.array([ax]) 
        
        for i in range(len(algorithmes)):
            mat_sizes[i].append(size)
            # Précision
            ax[i].plot(mat_sizes[i], accuracies[i], "-bo")
            ax[i].set_title("test " + algorithmes[i].__name__) 
            ax[i].set_xlabel("Nombre de ville")
            ax[i].set_ylabel("Précision") 
            ax[i].set_ylim(0, 1.1)
            
            # Coloration left
            ax[i].spines["left"].set_color('blue') 
            ax[i].yaxis.label.set_color('blue') 
            ax[i].tick_params(axis='y', colors='blue') 
            
            # Temps d'execution
            ax2 = ax[i].twinx()
            ax2.plot(mat_sizes[i], times[0], "-ro") 
            ax2.set_xlabel("Nombre de ville")
            ax2.set_ylabel("Temps d'execution (en ms)") 
            
            # Coloration right
            ax2.yaxis.label.set_color('red') 
            ax2.tick_params(axis='y', colors='red')
            ax2.spines["right"].set_color('red')
        
        fig.tight_layout()
        display.clear_output(wait=True) 
        plt.show() 
        
        
        #plt.show()
        #display.display(plt.gcf())
main_test([algo_genetique, algo_fourmis])